# NERCOME estimator to obtain covariance matrix

In [ ]:
import numpy as np
import itertools as it
import math
import matplotlib.pyplot as plt
%matplotlib inline

Generate data

In [ ]:
# Parameters
Nd = 4 # Number of random variables
Ns = 6 # Number of data realizations

In [ ]:
# Generate random matrix of size Nd x Ns
A = np.random.normal(0, 1, size=(Nd//2, Ns))
B = np.random.normal(0, np.sqrt(5), size=(Nd-Nd//2, Ns))
X = np.vstack((A, B))
# The resulting matrix X = (x_1, x_2, ..., x_Ns) consists of n column vectors,
# of which each x_i has length p, the upper half has variance 1 and the lower half has variance 5

#print(X)

The standard sample covariance estimator is given by
$ \hat{S} = \frac{1}{N_s-1} X X^T$

In [ ]:
S = 1/(Ns-1)*np.matmul(X, X.T)
print(S)

Then the covariance matrix $ \Sigma = \mathbb{E}(\hat{S}) $

In [ ]:
S_sum = np.zeros((Nd, Nd))
n = 10
for _ in range(n):
    A = np.random.normal(0, 1, size=(Nd//2, Ns))
    B = np.random.normal(0, np.sqrt(5), size=(Nd-Nd//2, Ns))
    X = np.vstack((A, B))
    S_sum += 1/(Ns-1)*np.matmul(X, X.T)
CovM = S_sum / n
print(CovM)

The NERCOME procedure divides the dataset into two subsamples, $X = (X_1, X_2)$, where $X_1$ is an $N_d \times s$ matrix and $X_2$ is an $N_d \times (N_s - s)$ matrix.

In [ ]:
def Q(s, X):
    Nd, Ns = X.shape
    assert s >= 2 and s <= Ns and math.floor(s) == s
    col_combos = list(it.combinations(range(Ns), s)) # Tuples of possible combinations of s out of Ns column indices

    Z_sum = np.zeros((Nd, Nd))
    S2_sum = np.zeros((Nd, Nd))

    for col_combo in col_combos:
        X1 = X[:, col_combo]
        X2 = X[:, np.delete(range(Ns), col_combo)]
        
        S1 = 1/(s-1)*np.matmul(X1, X1.T)
        S2 = 1/(Ns-s-1)*np.matmul(X2, X2.T)
        S2_sum += S2
        
        # Diagonalize S_i = U_i * D_i * U_i^T
        evals1, U1 = np.linalg.eigh(S1)
        D1 = np.diag(evals1)
        #evals2, U2 = np.linalg.eig(S2)
        #D2 = np.diag(evals2)
        
        # Verify that the matrix diagonalization is correct (up to absolute error of 1e^-10)
        S1_trial = np.matmul(np.matmul(U1, D1), U1.T)
        #S2_trial = np.matmul(np.matmul(U2, D2), U2.T)
        assert np.allclose(S1, S1_trial, 0, 1e-10)
        #assert np.allclose(S2, S2_trial, 0, 1e-10)
        
        # Compute estimator Z = U_1 * diag(U_1^T * S_2 * U_1) * U_1^T
        Z = np.matmul(np.matmul(U1, np.diag(np.diag(np.matmul(np.matmul(U1.T, S2), U1)))), U1.T)
        Z_sum += Z
    
    Z_avg = Z_sum / len(col_combos)
    S2_avg = S2_sum / len(col_combos)
    M = Z_avg - S2_avg
    
    return np.trace(np.matmul(M, M.T)) # Frobenius matrix norm

In [ ]:
# Parameters
Nd = 30 # Number of random variables
Ns = 14 # Number of data realizations

# Generate random matrix of size Nd x Ns
A = np.random.normal(0, 1, size=(Nd//2, Ns))
B = np.random.normal(0, np.sqrt(5), size=(Nd-Nd//2, Ns))
X = np.vstack((A, B))

In [ ]:
s = 10 # Trial value for s
print(Q(s, X))